## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())

Time taken by reduction operation using a function: 4.84 ms ± 70.5 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 24989.62328176169

Time taken by reduction operation using numpy.sum(): 13.3 µs ± 23.4 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 24989.62328176184 
 
Time taken by reduction operation using numpy.ndarray.sum(): 11.5 µs ± 3.85 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 24989.62328176184


a) Librería cupy: En la siguiente celda de código del notebook9 vamos a utilizar el paquete cupy para acelerar dicha operación de reducción. Como se ha explicado, la libreria cupy es una librería muy similar a la librería numpy específicamente diseñada para GPUs. De hecho, la mayoría de funciones que hay en numpy tienen el mismo nombre en cupy. Por tanto, de las 2 formas de hacer la suma de los elementos del array, por medio de la función reduc_operation y por medio de la función sum de la librería numpy, vamos a usar únicamente la función sum de la librería cupy.

Lo que tienes que hacer es modificar el notebook para crear el array en la GPU (usando las funciones de la librería cupy análogas a las de la librería numpy) y utilizar la función sum para calcular la suma de los elementos del array. Como la GPU ya es paralela, no tienes que paralelizar nada más.

In [3]:
import cupy as cp  # Importamos CuPy

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Configuración inicial
value = 5 * 10**4  # Tamaño del array

# Crear el array directamente en la GPU utilizando cupy.random.rand
X_gpu = cp.random.rand(value)  

# Para imprimir los primeros valores del array (traspasándolos a la CPU con .get())
# print(cp.asnumpy(X_gpu[:12]))

# Reducir operación en la GPU con cupy.sum()
tiempo = %timeit -r 2 -o -q cp.sum(X_gpu)

print("Time taken by reduction operation using cupy.sum():", tiempo)
print("Result of the sum using cupy.sum():", cp.sum(X_gpu), "\n")

# Alternativamente, usar el método .sum() del array creado por CuPy 
tiempo = %timeit -r 2 -o -q X_gpu.sum()

print("Time taken by reduction operation using cupy.ndarray.sum():", tiempo)
print("Result of the sum using cupy.ndarray.sum():", X_gpu.sum())

Time taken by reduction operation using cupy.sum(): 17.4 µs ± 25.4 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Result of the sum using cupy.sum(): 24950.91716312461 

Time taken by reduction operation using cupy.ndarray.sum(): 16.4 µs ± 31.2 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Result of the sum using cupy.ndarray.sum(): 24950.91716312461


b) Librería Numba: En la siguiente celda de código del notebook10 vamos a utilizar el paquete Numba para acelerar dicha operación de reducción. Como se ha explicado, la libreria Numba te permite crear ufuncs muy similares a las de la librería numpy que se pueden ejecutar en la GPU.

Lo que tienes que hacer es crear una ufunc que te permita hacer la reducción del array de forma análoga a las de la librería numpy, y utilizar la función sum para calcular la suma de los elementos del array. Como la GPU ya es paralela, no tienes que paralelizar nada más.

In [10]:
import numpy as np
from numba import cuda, float64

# Kernel de reducción en la GPU
@cuda.jit
def reduce_kernel(array, result):
    # Crear memoria compartida por bloque
    shared = cuda.shared.array(shape=256, dtype=float64)

    # Índice global y local
    tid = cuda.threadIdx.x
    gid = cuda.blockIdx.x * cuda.blockDim.x + tid

    # Copiar datos a memoria compartida (si están dentro del rango)
    if gid < array.size:
        shared[tid] = array[gid]
    else:
        shared[tid] = 0.0  # Si está fuera del rango, asignar 0
    cuda.syncthreads()

    # Reducción dentro del bloque
    step = cuda.blockDim.x // 2
    while step > 0:
        if tid < step:
            shared[tid] += shared[tid + step]
        step //= 2
        cuda.syncthreads()

    # El primer hilo escribe el resultado del bloque
    if tid == 0:
        result[cuda.blockIdx.x] = shared[0]

# Función de reducción completa usando GPU
def reduce_array_gpu(array):
    # Tamaño de bloques e inicialización
    threads_per_block = 256
    blocks_per_grid = (array.size + threads_per_block - 1) // threads_per_block

    # Arrays en la GPU
    d_array = cuda.to_device(array)
    d_result = cuda.device_array(blocks_per_grid, dtype=np.float64)

    # Primera reducción
    reduce_kernel[blocks_per_grid, threads_per_block](d_array, d_result)

    # Reducir el resultado en la CPU (cuando los bloques son pocos)
    h_result = d_result.copy_to_host()
    return np.sum(h_result)

# Configuración inicial
value = 5 * 10**4  # Tamaño del array

# Crear el array con NumPy
X = np.random.rand(value)

# Medir el tiempo de la reducción en la GPU
tiempo = %timeit -r 2 -o -q reduce_array_gpu(X)

print("Time taken by reduction operation using Numba kernel (GPU):", tiempo)
print("Result of the sum using Numba kernel (GPU):", reduce_array_gpu(X))

Time taken by reduction operation using Numba kernel (GPU): 409 µs ± 411 ns per loop (mean ± std. dev. of 2 runs, 1,000 loops each)
Result of the sum using Numba kernel (GPU): 25018.2432011578


c) Una vez hayas comprobado que funcionan los anteriores ejercicios, haz una modificación en el notebook que permita darle el valor del número de elementos por la línea de comandos al lanzar a ejecutar el notebook con el gestor de colas sbatch. Una vez hecho esto, crea el shell script que va a usar el comando sbatch para lanzar con SLURM a la cola bohr-gpu para medir el tiempo de ejecución en dicha cola. Llama a dicho fichero submit_reduc_oper-GPU_machine-name-login.sh11 Lánzalo con el valor de 5 ∗ 106, 5 ∗107 y 5∗108 elementos.

In [14]:
import argparse
import numpy as np
import cupy as cp
from numba import cuda, float64
import time
import sys

# Filtramos los argumentos pasados por Jupyter o IPython
if 'ipykernel_launcher' in sys.argv[0]:
    sys.argv = sys.argv[:1]  # Esto eliminará los argumentos adicionales

# Configuración inicial: agregar argparse para tomar el número de elementos desde la línea de comandos
parser = argparse.ArgumentParser(description="Reduction Operation GPU")
parser.add_argument('--value', type=int, default=5 * 10**4, help="Number of elements in the array")
args = parser.parse_args()

# Obtener el valor de los elementos
value = args.value  # Número de elementos a usar

# **Uso de CuPy para la reducción en la GPU**
X_gpu = cp.random.rand(value)

# Reducir operación en la GPU con cupy.sum()
tiempo = %timeit -r 2 -o -q cp.sum(X_gpu)
print(f"Time taken by reduction operation using cupy.sum(): {tiempo.best * 1e6:.2f} µs ± {tiempo.stdev * 1e6:.2f} µs per loop")
print(f"Result of the sum using cupy.sum(): {tiempo.best}\n")

# **Alternativa con cupy.ndarray.sum()**
tiempo = %timeit -r 2 -o -q X_gpu.sum()
print(f"Time taken by reduction operation using cupy.ndarray.sum(): {tiempo.best * 1e6:.2f} µs ± {tiempo.stdev * 1e6:.2f} µs per loop")
print(f"Result of the sum using cupy.ndarray.sum(): {tiempo.best}\n")

# **Kernel de reducción en la GPU para Numba**
@cuda.jit
def reduce_kernel(array, result):
    shared = cuda.shared.array(shape=256, dtype=float64)
    tid = cuda.threadIdx.x
    gid = cuda.blockIdx.x * cuda.blockDim.x + tid
    if gid < array.size:
        shared[tid] = array[gid]
    else:
        shared[tid] = 0.0
    cuda.syncthreads()

    step = cuda.blockDim.x // 2
    while step > 0:
        if tid < step:
            shared[tid] += shared[tid + step]
        step //= 2
        cuda.syncthreads()

    if tid == 0:
        result[cuda.blockIdx.x] = shared[0]

def reduce_array_gpu(array):
    threads_per_block = 256
    blocks_per_grid = (array.size + threads_per_block - 1) // threads_per_block

    d_array = cuda.to_device(array)
    d_result = cuda.device_array(blocks_per_grid, dtype=np.float64)

    reduce_kernel[blocks_per_grid, threads_per_block](d_array, d_result)
    h_result = d_result.copy_to_host()
    return np.sum(h_result)

# Medir el tiempo de la reducción en la GPU usando Numba
tiempo = %timeit -r 2 -o -q reduce_array_gpu(X_gpu)
print(f"Time taken by reduction operation using Numba kernel (GPU): {tiempo.best * 1e6:.2f} µs ± {tiempo.stdev * 1e6:.2f} µs per loop")
print(f"Result of the sum using Numba kernel (GPU): {tiempo.best}\n")

Time taken by reduction operation using cupy.sum(): 17.81 µs ± 0.07 µs per loop
Result of the sum using cupy.sum(): 1.7805099431425333e-05

Time taken by reduction operation using cupy.ndarray.sum(): 16.76 µs ± 0.02 µs per loop
Result of the sum using cupy.ndarray.sum(): 1.676220890134573e-05

Time taken by reduction operation using Numba kernel (GPU): 227.08 µs ± 0.01 µs per loop
Result of the sum using Numba kernel (GPU): 0.000227081635966897



d) Crea una nueva celda de texto debajo de la última celda de código para explicar los resultados obtenidos por los paquetes cupy y Numba usando la GPU.

En este ejercicio, se compararon dos librerías para optimizar una operación de reducción en la GPU: CuPy y Numba. CuPy demostró ser la opción más rápida, con tiempos de ejecución de 17.81 µs y 16.76 µs para las funciones cupy.sum() y cupy.ndarray.sum(), respectivamente. Esto refleja la eficiencia de CuPy, que está diseñada específicamente para trabajar con operaciones de arrays en la GPU, ofreciendo tiempos de ejecución bajos y estables.

Por otro lado, Numba, aunque también eficiente, mostró tiempos de ejecución más altos, con 227.08 µs por iteración al utilizar un kernel personalizado para la reducción. Esta diferencia se debe a que Numba ofrece mayor flexibilidad para crear operaciones personalizadas en la GPU, pero la sobrecarga de la compilación y la gestión del kernel hace que no sea tan rápido como las funciones optimizadas de CuPy.